
# Introduction
This notebook will:
1. Scrape and clean all press statements from last 3 years (ending from 13 March 2024)
2. Chunk the documents into passages
3. Load them into our Elasticsearch Docker container
4. Test out some queries.

The next step will be the LLM testing notebook, where we will use the loaded Elasticsearch Docker container to do RAG.

#pip installs

In [1]:
%pip install elasticsearch pandas beautifulsoup4 langchain tiktoken

Defaulting to user installation because normal site-packages is not writeable
  Using cached tiktoken-0.6.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     -------------------------------------- 107.3/107.3 kB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl.metadata (32 kB)
  Using cached typing_inspect-0.9.0-py3-none-

#imports

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import NavigableString
import time
import re
import datetime
from langchain.text_splitter import RecursiveCharacterTextSplitter
from elasticsearch import Elasticsearch, helpers
import json
import pickle

#Scraping and Cleaning MFA Press statements

In [5]:
def scrape(base_url, startdate, enddate):
    headers = {'User-Agent': 'Mozilla/5.0'}
    all_posts_info = []

    pages_scraped = 0
    start_time = time.time()

    # Scraping loop
    page = 1
    while True:
        url = f"{base_url}?keyword=&country=&startdate={startdate}&enddate={enddate}&topic=F5B66A618AEB4477A3E97996C9D05255&page={page}" # the value after topic represents Press Statements Only
        print(f"Scraping URL: {url}")
        response = requests.get(url, headers=headers)
        # time.sleep(1)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            statements = soup.select('body > div.off-canvas-wrapper > div > div > section > div.press-release > div > div > div.strip2')# web index

            if not statements:
                print(f"No more statements found. Exiting...")
                break

            for statement in statements:
                title_element = statement.select_one('div.sec2 > h3 > a')
                meta_element = statement.select_one('div.sec2 > p.strip2__labels') # the parent of 3 elements, datetime, type and country.
                date = meta_element.select_one('span.date').get_text(strip=True)
                countries = []
                for element in meta_element:
                    if isinstance(element, str):  # Skip any string elements, like newlines.
                        continue
                    if (element.get_text(strip=True) in ["Press Statements", "Spokeperson's Comments", "Transcripts", "Outcome Documents"]) or not (element.get_text(strip=True).isalpha()):  # Skip known labels.
                        continue  # Skip collecting details about its document type
                    countries.append(element.get_text(strip=True))
                country = ", ".join(countries)



                # time.sleep(1)
                statement_response = requests.get(title_element['href'], headers=headers)
                if statement_response.status_code == 429:
                    print(f"Rate limit hit, saving collected data and exiting...")
                    return all_posts_info

                statement_soup = BeautifulSoup(statement_response.text, 'html.parser')

                  # Select the content container
                content_container = statement_soup.select_one('body > div.off-canvas-wrapper > div > div > section > div.innerpage2 > div > div > div:nth-child(1)')

                # Assume 'content_container' is a BeautifulSoup object containing the parsed HTML of a content container.

                if content_container:
                    content_pieces = []
                    for paragraph in content_container.select('div, p'):
                        paragraph_text = ''
                        for elem in paragraph.descendants:
                            if isinstance(elem, NavigableString):
                                if elem and any(char.isalpha() or char in "()[]" for char in elem):
                                    # Append text with a space only if there's already text accumulated.
                                    paragraph_text += elem
                            elif elem.name == 'br':  # Check if the element is a line break
                                paragraph_text += ' '  # Add a space for a line break

                        # Collapse multiple whitespace into single spaces for the entire paragraph
                        paragraph_text = ' '.join(paragraph_text.split())

                        # Append the entire paragraph text to content pieces with a newline
                        if paragraph_text:  # Make sure not to append empty paragraphs
                            content_pieces.append(paragraph_text)

                    # Join all paragraphs with a newline
                    content = '\n'.join(content_pieces)

                    print(f"Page {page}: {date}, {country}, {len(content)} content length.")

                    # Add the post info including code length
                    all_posts_info.append({
                        'url': title_element['href'],
                        'title': title_element.get_text(strip=True),
                        'date': date,
                        'country': country,
                        'content': content
                    })

            pages_scraped += 1
            elapsed_time = time.time() - start_time
            avg_time_per_page = elapsed_time / pages_scraped

            # Since 'total_pages' is not available, we cannot calculate 'pages_left' and 'estimated_time_left' directly.
            # Instead, we will display the average time per page and the number of pages scraped so far.
            print(f"\nPage {page} scraped. Total pages scraped so far: {pages_scraped}. Average time per page: {avg_time_per_page:.2f} seconds.\n")
            page += 1

        elif response.status_code == 429:
            print(f"Rate limit hit, saving collected data and exiting...")
            return all_posts_info
        else:
            print(f"Error fetching page {page}, Status Code: {response.status_code}")
            return all_posts_info

    elapsed_time = time.time() - start_time
    print(f"\nScraping completed. {pages_scraped} pages scraped in {elapsed_time:.2f} seconds.\n")

    return all_posts_info

base_url = 'https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos'
#date: mm/dd/yyyy
data = scrape(base_url, '01/01/2021', '08/05/2024')
print(len(data), "statements collected.")
df = pd.DataFrame(data)

Scraping URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos?keyword=&country=&startdate=01/01/2021&enddate=08/05/2024&topic=F5B66A618AEB4477A3E97996C9D05255&page=1
Page 1: 08 May 2024, 23:15, , 567 content length.
Page 1: 08 May 2024, 23:02, , 1769 content length.
Page 1: 08 May 2024, 19:44, , 2238 content length.
Page 1: 08 May 2024, 19:40, , 2138 content length.
Page 1: 08 May 2024, 19:35, Israel, 986 content length.

Page 1 scraped. Total pages scraped so far: 1. Average time per page: 2.03 seconds.

Scraping URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos?keyword=&country=&startdate=01/01/2021&enddate=08/05/2024&topic=F5B66A618AEB4477A3E97996C9D05255&page=2
Page 2: 07 May 2024, 21:16, , 1283 content length.
Page 2: 07 May 2024, 20:02, Malaysia, 2404 content length.
Page 2: 07 May 2024, 19:36, , 1499 content length.
Page 2: 05 May 2024, 18:20, Malaysia, 1208 content length.
Page 2: 05 May 2024, 18:11, Poland, 2047 content length

In [8]:
import json

# Save the scraped data into a JSON file
# with open('mfa_statements.json', 'w') as file:
#     json.dump(data, file, indent=4)

with open('mfa_statements.json', 'r') as file:
    data = json.load(file)

print(len(data), "statements saved and loaded.")
docs = data

875 statements saved and loaded.


# Chunking
I would like to chunk the documents so that the most relevant parts of the document can be shown to the chatbot, however, I would also like to know which document the chunk is from, so that I can reference it later.

In [9]:
# Convert the 'date' column to datetime first

# Adjust the date parsing to handle different date formats
for doc in docs:
    try:
        # Attempt to parse the date string into a datetime object using the expected format
        date_time_obj = datetime.datetime.strptime(doc['date'], '%d %b %Y, %H:%M')
    except ValueError:
        # If the first format fails, try parsing with an alternative format
        date_time_obj = datetime.datetime.fromisoformat(doc['date'])

    # Convert the datetime object to a string in ISO 8601 format as required by Elasticsearch
    doc['date'] = date_time_obj.isoformat()

df = pd.DataFrame(docs)

metadata = []
content = []
import random

# Iterate over the DataFrame to gather content and metadata
for index, row in df.iterrows():
    # Add the document content to the content list
    content.append(row['content'])

    # Create a metadata dictionary for this document, including the new fields
    meta = {
        "id": random.randint(1000, 9999),  # Generate a random ID for each document
        "url": row['url'],
        "title": row['title'],
        "date": row['date'],
        "country": row['country'],
    }

    # Add the metadata dictionary to the metadata list
    metadata.append(meta)

# Initialize the text splitter with desired chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=256)

# Create chunks (passages) with associated metadata
docs = text_splitter.create_documents(content, metadatas=metadata)

print(f"Split {len(df)} documents into {len(docs)} passages")

Split 875 documents into 8947 passages


In [12]:
import pickle

with open('mfa_chunks.pkl', 'wb') as outp:  # 'wb' for writing in binary mode
    pickle.dump(docs, outp, pickle.HIGHEST_PROTOCOL)



In [12]:
# Open the file in binary read mode ('rb') and load its contents into the variable `docs`
with open('mfa_chunks.pkl', 'rb') as inp:
    docs = pickle.load(inp)
docs[0].page_content

'QUESTION Dr Wan RizalTo ask the Minister for Foreign Affairs whether the Ministry can provide an update on the measures currently in place to ensure the safety of Singaporeans residing in or travelling to the Middle East, particularly students, in light of recent military actions reported between Israel and Iran. REPLY 1 The question was addressed in MFA’s reply to a similar Question No. 5968 in Notice Paper No. 2733 of 2024, submitted by Member Zhulkarnain Abdul Rahim for the parliamentary sitting on 7 May'

In [13]:
# Prepare to load into Elasticsearch
final_docs = []
for doc in docs:
  meta = dict(doc.metadata)
  final_docs.append({
    "title": meta['title'],
    "url": meta['url'],
    "date": meta['date'],
    "country": meta['country'],
    "content": doc.page_content
  })

# Delete the original 'docs' list to free up memory
del docs


## Load into Elasticsearch
We used docker, enabled the security configuration which was quite **painful** to work with, but it would ensure that documents are secure.

In [14]:
# Connect to client (make sure Elasticsearch is running and listening on the specified address)
client = Elasticsearch("https://localhost:9200/",
                       basic_auth=("elastic", "*Td3NjQLaQPQQ*OYc5ES")) #yes the password is here, but its useless to you since its a locally hosted elasticsearch instance
try:
    client.indices.delete(index='article')
except:
    pass

In [19]:
with open('mfa_chunks_es.json', 'w') as file:
  save_to_json = json.dumps(final_docs)
  file.write(save_to_json)

In [15]:
with open('mfa_chunks_es.json', 'r') as file:
  final_docs = json.load(file)
print(final_docs[-1]['title'])

Condolences from Singapore Leaders on the Crash of Sriwijaya Air Flight SJ182


In [16]:
# Map the index's columns
index_name = "article"
index_mappings = {
  "mappings": {
    "properties": {
      "date": {"type": "date"},
      "title": {"type": "text"},
      "url": {"type": "text"},
      "country": {"type": "text"},
      "content": {"type": "text"},
    }
  }
}

if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name, body=index_mappings)


actions = [
    {
        "_index": index_name,
        "_source": {
            "title": doc['title'],
            "url": doc['url'],
            "date": doc['date'],
            "country": doc['country'],
            "content": doc['content']
        }
    } for doc in final_docs
]

# Use the helpers.bulk() to add the documents to the index
resp = helpers.bulk(client, actions)
print(f"""Before: {client.count(index=index_name)['count']} documents 
Added: +{resp[0]} documents
Now: {int(resp[0])+int(client.count(index=index_name)['count'])} documents.""")
# its not accurate. make a GET request to ES /index/_search and check the hits.total.value



Before: 4473 documents
Added: +8947 documents
Now: 13420 documents.


## Testing Elasticsearch & Abstraction

Searching this way, Elasticsearch uses the BM25 algorithm by default, which is a extension of the TF-IDF algorithm with improvements that better handle the variation on document length and saturation of term frequency.

If you just want to ingest data, this is the end. The rest is just playing around with retrieving data.

In [30]:
def es_search(client, search_body):
    pretty_result = ""

    # Perform the search query
    response = client.search(index="article", body=search_body)

    # Extract the relevant information
    hits = response['hits']['hits']
    total_hits = response['hits']['total']['value']

    # Build a readable result string
    pretty_result += f"Total documents found: {total_hits}\n"
    for i, hit in enumerate(hits, start=1):
        source = hit['_source']
        pretty_result += (
            f"Document {i}:\n"
            f"  Title: {source['title']}\n"
            f"  Date: {source['date']}\n"
            f"  URL: {source['url']}\n"
            f"  Country: {source['country']}\n"
            f"  Content: {source['content']}\n\n"  # Truncate content for display
        )

    return pretty_result.strip()

In [31]:
# search based on date fields
print(es_search(client, {
    "query": {
      "range": {
        "date": {
            "gte": "2024-03-03T00:00:00",  # March 3rd, 2024
            "lte": "2024-03-05T23:59:59"   # March 5th, 2024
            }}}}))

Total documents found: 71
Document 1:
  Title: 9th Singapore-Australia Annual Leaders' Meeting, 5 March 2024
  Date: 2024-03-05T10:13:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2024/03/20240305-9th-SA-ALM
  Country: Australia
  Content: Prime Minister Lee Hsien Loong and Australian Prime Minister Anthony Albanese held the 9th Singapore-Australia Annual Leaders’ Meeting in Melbourne today. The Prime Ministers held a Joint Press Conference after their Meeting.

Document 2:
  Title: 9th Singapore-Australia Annual Leaders' Meeting, 5 March 2024
  Date: 2024-03-05T10:13:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2024/03/20240305-9th-SA-ALM
  Country: Australia
  Content: The Prime Ministers reaffirmed the excellent state of bilateral relations and noted the good progress in bilateral cooperation across all six pillars of the Singapore-Australia Comprehensive Strategic Partnership (CSP), namely Economics and Trade,

In [32]:
# search based on content fields
print(es_search(client, {
    "query": {
      "match": {
        "content": "Covid 19"}}}))

Total documents found: 624
Document 1:
  Title: Minister for Foreign Affairs Dr Vivian Balakrishnan’s Written Replies to Parliamentary Questions, 26 July 2021
  Date: 2021-07-26T23:00:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2021/07/20210726-Written-Replies-to-PQs
  Country: Cambodia, Indonesia, Laos, Malaysia, Myanmar, Philippines, Singapore, Thailand, Vietnam
  Content: Turning to ASEAN, Singapore has actively contributed to ASEAN’s efforts to mitigate the effects of COVID-19. At the outset of the COVID-19 pandemic, we pushed for ASEAN to coordinate its response and this resulted in the establishment of the ASEAN Coordinating Council Working Group on Public Health Emergencies. In 2020, the Working Group agreed to, among others, establish the COVID-19 ASEAN Response Fund.

Document 2:
  Title: Dose Sharing Arrangement of COVID-19 Vaccines Between Singapore and Australia to Support Efforts to Combat COVID-19
  Date: 2021-08-31T11:25:00
  URL: ht

In [33]:
# search based on phrase
print(es_search(client, {
    "query": {
      "match_phrase": {
        "content": "Covid-19 vaccination"
        }}}))

Total documents found: 3
Document 1:
  Title: Minister for Foreign Affairs Dr Vivian Balakrishnan's Written Reply to Parliamentary Question on COVID-19 Vaccination rates of Singaporean students studying abroad and efforts to ensure their safety, 3 November 2021
  Date: 2021-11-03T21:54:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2021/11/20211103-PQ
  Country: Singapore
  Content: QUESTION
Dr Wan Rizal To ask the Minister for Foreign Affairs (a) what are the COVID-19 vaccination rates of Singaporean students studying abroad; and (b) what are the current efforts in ensuring the safety of our students overseas in a COVID-19 world.
REPLY
On part (a) of the query, we do not have statistics on the COVID-19 vaccination rates of Singaporean students who are based abroad.

Document 2:
  Title: COVID-19 Vaccination Channels for Overseas Singaporeans
  Date: 2021-09-11T18:58:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/202

In [34]:
# search using multi-match queries
# must be from SG, must not be from MY, should match phrase "Covid-19 vaccination"
print(es_search(client, {
    "query": {
        "bool": {
            "must": [
                {"match":{
                    "country": "Singapore"}}],
            "must_not": [
                {"match":{
                    "country": "Malaysia"}}],
            "should": [
                {"match_phrase": {
                    "content": "Covid-19 vaccination"}}]
            }}}))

Total documents found: 141
Document 1:
  Title: Minister for Foreign Affairs Dr Vivian Balakrishnan's Written Reply to Parliamentary Question on COVID-19 Vaccination rates of Singaporean students studying abroad and efforts to ensure their safety, 3 November 2021
  Date: 2021-11-03T21:54:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2021/11/20211103-PQ
  Country: Singapore
  Content: QUESTION
Dr Wan Rizal To ask the Minister for Foreign Affairs (a) what are the COVID-19 vaccination rates of Singaporean students studying abroad; and (b) what are the current efforts in ensuring the safety of our students overseas in a COVID-19 world.
REPLY
On part (a) of the query, we do not have statistics on the COVID-19 vaccination rates of Singaporean students who are based abroad.

Document 2:
  Title: Transcript of Minister for Foreign Affairs Dr Vivian Balakrishnan's Live Interview on Channel NewsAsia's "Singapore Tonight", 29 February 2024
  Date: 2024-03-01T00

# Another layer of abstraction for our LLM
Using the base of the multi-query, we open up possibilities to abstract our query making. This is important, so as to avoid any sort of syntax error created by our model.

In [49]:
def ESrch(client, must_query=None, must_not_query=None, should_query=None):
    bool_query = {}
    if must_query:
        bool_query["must"] = must_query if isinstance(must_query, list) else [must_query]
    if must_not_query:
        bool_query["must_not"] = must_not_query if isinstance(must_not_query, list) else [must_not_query]
    if should_query:
        bool_query["should"] = should_query if isinstance(should_query, list) else [should_query]
    response = client.search(index="article", body={"query": {"bool": {k: v for k, v in bool_query.items() if v}}})
    hits = response['hits']['hits']
    pretty_result = "\n".join([
        f"Total documents found: {response['hits']['total']['value']}"] + [
        f"Document {i+1}:\n  Title: {hit['_source'].get('title', 'N/A')}\n  Date: {hit['_source'].get('date', 'N/A')}\n  URL: {hit['_source'].get('url', 'N/A')}\n  Country: {hit['_source'].get('country', 'N/A')}\n  Content: {hit['_source'].get('content', 'N/A')}\n"
        for i, hit in enumerate(hits)])
    result_hits = [hit['_source'] for hit in hits]
    return result_hits, pretty_result

In [53]:
# search for date fields
must = {"range": {"date": {"gte": "2024-03-03T00:00:00"}}} # mandatory positive result
must_not = None # mandatory negative result
should = None # boosts the matched result
hits, result = ESrch(client, must)
print(result)

Total documents found: 669
Document 1:
  Title: Minister for Foreign Affairs Dr Vivian Balakrishnan's Written Reply to Parliamentary Question on Whether the Ministry Can Provide an Update on the Measures Currently in Place to Ensure the Safety of Singaporeans Residing in or Travelling to the Middle East, Particularly Students, in Light of Recent Military Actions Reported between Israel and Iran, 8 May 2024
  Date: 2024-05-08T23:15:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2024/05/20240508-Written-PQ-on-Safety-of-Singaporeans-in-ME
  Country: 
  Content: QUESTION Dr Wan RizalTo ask the Minister for Foreign Affairs whether the Ministry can provide an update on the measures currently in place to ensure the safety of Singaporeans residing in or travelling to the Middle East, particularly students, in light of recent military actions reported between Israel and Iran. REPLY 1 The question was addressed in MFA’s reply to a similar Question No. 5968 in N

In [51]:
# search using multi-match queries
must = [{"match": {"country": "Singapore"}}, {"range": {"date": {"gte": "2023-03-03T00:00:00"}}}]
must_not = {"match": {"country": "Malaysia"}}
should = {"match_phrase": {"content": "Covid-19 vaccination"}}
hits, results = ESrch(client,must,must_not,should)
print(results)

Total documents found: 112
Document 1:
  Title: Transcript of Minister for Foreign Affairs Dr Vivian Balakrishnan's Live Interview on Channel NewsAsia's "Singapore Tonight", 29 February 2024
  Date: 2024-03-01T00:40:00
  URL: https://www.mfa.gov.sg/Newsroom/Press-Statements-Transcripts-and-Photos/2024/02/240229_Minister-COS-CNA-Interview
  Country: Singapore
  Content: Dawn Tan (CNA): The Israel-Hamas conflict featured very strongly in Parliament today. What would you say to those who perhaps are still unconvinced about Singapore’s neutrality in this matter?
Minister Vivian Balakrishnan: The first thing I would say is that the real question is not neutrality. The real question is, “What is in Singapore’s national interest?”. We had an extensive discussion in Parliament today, and I would say the following.

Document 2:
  Title: Transcript of Minister for Foreign Affairs Dr Vivian Balakrishnan's Live Interview on Channel NewsAsia's "Singapore Tonight", 29 February 2024
  Date: 2024-03-0